In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [2]:
# export
import torch

In [3]:
# export
from torch.nn import functional as F

In [4]:
# export
from torch import tensor

In [5]:
from matplotlib import pyplot as plt

In [6]:
from IPython.core import debugger as idb

In [7]:
# export
import numpy as np

In [8]:
# export
import math

# functions

## dice_coef

In [9]:
# export
def dice_coef(input, target):
    smooth = 1.
    
    pred = input.sigmoid()
    target = target.float()
    
    return ((2. * (pred * target).sum() + smooth) / (pred.sum() + target.sum() +smooth))

## dice_loss

In [10]:
# export
def dice_loss(input, target):
    return 1 - dice_coef(input, target)

## weighted_bce

In [11]:
# export
def weighted_bce(input, target, pos_weight=1):
    """
    pos_weight: positive weight relative to negative weight(which is 1)
    """
    mask = (target>0).float()
    weight = (mask*pos_weight + (1-mask))
    weight = weight/weight.sum()*mask.numel()
    return F.binary_cross_entropy_with_logits(input,target,weight=weight)

## balance_bce

In [12]:
# export
def balance_bce(input, target, balance_ratio=1):
    """
    Auto adjust positive/negative ration as set by balance_ratio.
    """
    mask = (target>0).float()
    posN = mask.sum()
    negN = (1-mask).sum()
    pos_weight = balance_ratio*negN/posN
    
    return weighted_bce(input, target, pos_weight)

## mask_iou

In [13]:
# export
def mask_iou(input, target):
    """
    iou for segmentation
    """
    pred_mask = input>0
    target_mask = target>0
    
    i = (pred_mask&target_mask).float().sum()
    u = (pred_mask|target_mask).float().sum()
    return i/u

# test

## dice_coef

In [14]:
x = torch.randn((4,3,512,512))
y = torch.randint_like(x,0,2)

dice_coef(x,y)

tensor(0.4999)

## dice_loss

In [15]:
x = torch.randn((4,3,512,512))
y = torch.randint_like(x,0,2)

dice_loss(x,y)

tensor(0.5001)

## weighted_bce

In [16]:
x = torch.randn((4,3,512,512))+2 # 让x平均值为2，则概率sigmoid(x)的平均值>0.5，所以正例的损失小，负例的损失大
y = torch.randint_like(x,0,2)

In [17]:
weighted_bce(x,y)

tensor(1.1828)

In [18]:
weighted_bce(x,y,2)

tensor(0.8495)

In [19]:
weighted_bce(x,y,20)

tensor(0.2780)

看到pos_weight越大，则加权损失越靠近正例的损失，即变小。

## balance_bce

In [20]:
x = torch.randn((4,3,512,512))+2 # 让x平均值为2，则概率sigmoid(x)的平均值>0.5，所以正例的损失小，负例的损失大
y = torch.randint_like(x,0,3) # 让正例占2/3，负例1/3

In [21]:
weighted_bce(x,y), balance_bce(x,y,2)

(tensor(0.1831), tensor(0.1829))

In [22]:
weighted_bce(x,y,0.5), balance_bce(x,y,1)

(tensor(0.6831), tensor(0.6828))

## mask_iou

In [23]:
x = torch.randn((4,3,512,512))
y = torch.randint_like(x,0,2)
mask_iou(x,y)

tensor(0.3333)

x有50%正，y有50%正，二者交集为25%，二者并集为75%，所以交集/并集=1/3

# export

In [24]:
!python notebook2script.py --fname 'loss_metrics.ipynb' --outputDir './exp/'

Converted loss_metrics.ipynb to exp/nb_loss_metrics.py
